In [3]:
import os
import pandas as pd
import numpy as np
import cv2
import requests
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

In [4]:
# Define SE Block
def se_block(input_tensor, reduction_ratio=16):
    channels = input_tensor.shape[-1]
    se_shape = (1, 1, channels)

    squeeze = layers.GlobalAveragePooling2D()(input_tensor)
    excitation = layers.Dense(channels // reduction_ratio, activation='relu')(squeeze)
    excitation = layers.Dense(channels, activation='sigmoid')(excitation)
    excitation = layers.Reshape(se_shape)(excitation)

    return layers.multiply([input_tensor, excitation])

In [5]:

# Load dataset from CSV
def load_dataset(file_path):
    data = pd.read_csv(file_path, header=None)
    images = data.iloc[:, ::5].values  # Image URLs
    bboxes = data.iloc[:, 1:4].values.reshape(-1, 3, 4)  # Bounding boxes
    labels = data.iloc[:, 15:].values.reshape(-1, 3)  # Labels
    return images, bboxes, labels

In [6]:
# Download and preprocess images
def preprocess_data(image_urls, bboxes, labels, img_size=(224, 224)):
    processed_images = []
    processed_labels = []
    for img_set, bbox_set, label_set in zip(image_urls, bboxes, labels):
        for img_url, bbox, label in zip(img_set, bbox_set, label_set):
            try:
                # Download image
                resp = requests.get(img_url, stream=True)
                img_arr = np.asarray(bytearray(resp.content), dtype=np.uint8)
                img = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)

                # Crop bounding box
                h, w, _ = img.shape
                x_min, x_max = int(bbox[0] * w), int(bbox[1] * w)
                y_min, y_max = int(bbox[2] * h), int(bbox[3] * h)
                cropped_img = img[y_min:y_max, x_min:x_max]

                # Resize and normalize
                resized_img = cv2.resize(cropped_img, img_size) / 255.0
                processed_images.append(resized_img)
                processed_labels.append(label)
            except Exception as e:
                print(f"Error processing {img_url}: {e}")

    return np.array(processed_images), np.array(processed_labels)

In [7]:
# Build CNN model with SE Blocks
def build_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = se_block(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = se_block(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [18]:
# Check file existence
train_csv_path = "C:\\Users\\ASUS\\Desktop\\Facial Emotion Comparison Detection\\faceexp-comparison-data-train-public.csv"
test_csv_path = "C:\\Users\\ASUS\\Desktop\\Facial Emotion Comparison Detection\\faceexp-comparison-data-test-public.csv"

if not os.path.exists(train_csv_path):
    print(f"Train file not found: {train_csv_path}")
if not os.path.exists(test_csv_path):
    print(f"Test file not found: {test_csv_path}")

In [25]:
def load_dataset(file_path):
    try:
        # Load CSV
        data = pd.read_csv(file_path, header=None, engine='python', on_bad_lines='skip')
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None, None, None

    print(f"Dataset shape: {data.shape}")

    # Extract image URLs
    images = data.iloc[:, ::5].values  # Assuming URLs are in every 5th column

    # Extract bounding boxes
    try:
        bboxes = data.iloc[:, 1:13].values.reshape(-1, 3, 4)  # Adjust indices if needed
        print(f"BBoxes shape after reshaping: {bboxes.shape}")
    except ValueError as e:
        print(f"Error reshaping bounding boxes: {e}")
        return None, None, None

    # Extract labels
    labels = data.iloc[:, 15:].values
    print(f"Labels shape before reshaping: {labels.shape}")
    try:
        num_samples = labels.shape[0] // 3
        labels = labels.reshape(num_samples, 3)
    except ValueError as e:
        print(f"Error reshaping labels: {e}")
        return None, None, None

    return images, bboxes, labels


In [26]:
# Load datasets
train_images, train_bboxes, train_labels = load_dataset(train_csv_path)
test_images, test_bboxes, test_labels = load_dataset(test_csv_path)


Dataset shape: (446536, 28)
BBoxes shape after reshaping: (446536, 3, 4)
Labels shape before reshaping: (446536, 13)
Error reshaping labels: cannot reshape array of size 5804968 into shape (148845,3)
Dataset shape: (50845, 28)
BBoxes shape after reshaping: (50845, 3, 4)
Labels shape before reshaping: (50845, 13)
Error reshaping labels: cannot reshape array of size 660985 into shape (16948,3)


In [29]:
 # Preprocess data
X_train, y_train = preprocess_data(train_images, train_bboxes, train_labels)
X_test, y_test = preprocess_data(test_images, test_bboxes, test_labels)

TypeError: 'NoneType' object is not iterable